In [8]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("reservas_simuladas.db")

### 1. Ingresos por país

In [9]:
query_1 = '''
SELECT 
    pais_cliente,
    SUM(monto_reserva) AS ingresos_totales
FROM reservas
WHERE estado_reserva = 'completada'
GROUP BY pais_cliente
ORDER BY ingresos_totales DESC;
'''
df1 = pd.read_sql_query(query_1, conn)
df1

,pais_cliente,ingresos_totales
0,México,263615.02
1,Canadá,200873.07
2,EEUU,162425.20
3,Alemania,141587.90
4,Brasil,129382.58
5,España,103219.66


### 2. Duración promedio de estancia por país

In [10]:
query_2 = '''
SELECT 
    pais_cliente,
    AVG(julianday(fecha_checkout) - julianday(fecha_checkin)) AS duracion_promedio
FROM reservas
WHERE estado_reserva = 'completada'
GROUP BY pais_cliente
ORDER BY duracion_promedio DESC;
'''
df2 = pd.read_sql_query(query_2, conn)
df2

,pais_cliente,duracion_promedio
0,España,4.202614
1,Brasil,4.042781
2,Canadá,3.898955
3,México,3.874016
4,EEUU,3.860262
5,Alemania,3.839024


### 3. Estacionalidad de reservas

In [11]:
query_3 = '''
SELECT 
    pais_cliente,
    strftime('%m', fecha_reserva) AS mes,
    COUNT(*) AS total_reservas
FROM reservas
GROUP BY pais_cliente, mes
ORDER BY pais_cliente, total_reservas DESC;
'''
df3 = pd.read_sql_query(query_3, conn)
df3

,pais_cliente,mes,total_reservas
0,Alemania,05,46
1,Alemania,01,37
2,Alemania,06,33
3,Alemania,03,33
4,Alemania,04,31
...,...,...,...
67,México,07,33
68,México,09,32
69,México,10,24
70,México,12,23


### 4. Preferencias por tipo de habitación

In [12]:
query_4 = '''
SELECT 
    pais_cliente,
    tipo_habitacion,
    COUNT(*) AS cantidad_reservas
FROM reservas
GROUP BY pais_cliente, tipo_habitacion
ORDER BY pais_cliente, cantidad_reservas DESC;
'''
df4 = pd.read_sql_query(query_4, conn)
df4

,pais_cliente,tipo_habitacion,cantidad_reservas
0,Alemania,estándar,132
1,Alemania,económica,113
2,Alemania,lujo,59
3,Brasil,económica,108
4,Brasil,estándar,105
5,Brasil,lujo,49
6,Canadá,estándar,155
7,Canadá,económica,154
8,Canadá,lujo,75
9,EEUU,estándar,136


### 5. Tasa de cancelación por país

In [13]:
query_5 = '''
SELECT 
    pais_cliente,
    ROUND(
        SUM(CASE WHEN estado_reserva = 'cancelada' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 
        2
    ) AS tasa_cancelacion_pct
FROM reservas
GROUP BY pais_cliente
ORDER BY tasa_cancelacion_pct DESC;
'''
df5 = pd.read_sql_query(query_5, conn)
df5

,pais_cliente,tasa_cancelacion_pct
0,Alemania,20.72
1,Brasil,19.47
2,México,18.25
3,Canadá,17.45
4,España,17.22
5,EEUU,16.83


In [14]:
conn.close()